In [1]:
import pandas as pd
import requests
from datetime import datetime
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime

# Excel 檔案名稱
FILENAME = "investment_tracking.xlsx"

# 初始化交易紀錄 DataFrame
columns = ['日期', '投資類型', '投資項目', '數量_or_外幣金額', '買進匯率','價格(台幣)']
if os.path.exists(FILENAME):
    transactions = pd.read_excel(FILENAME)
else:
    transactions = pd.DataFrame(columns=columns)

##### 黃金function

In [31]:
# 取得黃金即時價格
def get_gold_price():
    try:
        # Alpha Vantage 的 API URL
        url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=XAUUSD&apikey=T4QJDQEMPG0ED9A3&datatype=json"
        response = requests.get(url)
        data = response.json()
        
        # 獲取最近的日期
        latest_date = max(data["Time Series (Daily)"].keys())
        
        # 提取該日期的收盤價
        gold_price = float(data["Time Series (Daily)"][latest_date]["4. close"])
        
        print(f"黃金價格 (日期: {latest_date}): {gold_price} USD")
        return gold_price
    except Exception as e:
        print("取得黃金價格時出現錯誤：", e)
        return None

# 取得歷史黃金價格
def get_historical_gold_price(date):
    
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=XAUUSD&apikey=T4QJDQEMPG0ED9A3&datatype=json"
        response = requests.get(url)
        date = response.json()
        
        if "Time Series (Daily)" in date:
            historical_data = date["Time Series (Daily)"]
            if date in historical_data:
                gold_price = float(historical_data[date]["4. close"])  # 收盤價
                print(f"黃金價格 (日期: {date}): {gold_price} USD")
                return gold_price
            else:
                print(f"無法找到 {date} 的金價。")
                return None
        else:
            print("無法取得歷史金價:", date.get("Note", "Unknown error"))
            return None
    except Exception as e:
        print("取得歷史金價時出現錯誤：", e)
        return None


##### 外匯function

In [18]:
# 取得當前匯率
def get_exchange_rate(base_currency):
    try:
        # 這裡以 exchangeratesapi.io 為例，請替換成合適的API
        url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={base_currency}&to_symbol=TWD&apikey=T4QJDQEMPG0ED9A3&outputsize=full"
        response = requests.get(url)
        data = response.json()
        if base_currency in data['rates']:
            return data['rates'][base_currency]
        else:
            print(f"無法取得 {base_currency} 的匯率。")
            return None
    except Exception as e:
        print("取得匯率時出現錯誤：", e)
        return None
    
# 取得歷史匯率
def get_historical_exchange_rate(base_currency,  date):
    try:
        # Alpha Vantage API 的 URL
        url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={base_currency}&to_symbol=TWD&apikey=T4QJDQEMPG0ED9A3&outputsize=full"
        response = requests.get(url)
        data = response.json()

        # 檢查是否成功獲取數據
        if 'Time Series FX (Daily)' in data:
            time_series = data['Time Series FX (Daily)']
            # 轉換日期格式
            
            if date in time_series:
                # 提取對應日期的匯率
                exchange_rate = float(time_series[date]['4. close'])
                return exchange_rate
            else:
                print(f"無法獲取 {base_currency} 和 TWD 的匯率，日期可能不存在。")
                return None
        else:
            print("無法獲取數據，請檢查 API 金鑰或其他設置。")
            return None

    except Exception as e:
        print("取得匯率時出現錯誤：", e)
        return None



In [35]:
get_gold_price()
get_historical_gold_price('2024-10-16')
get_exchange_rate('USD', '2024-10-16')
# get_historical_exchange_rate('USD', '2024/10/16')


取得黃金價格時出現錯誤： 'Time Series (Daily)'
無法取得歷史金價: Unknown error
取得匯率時出現錯誤： Expecting value: line 1 column 1 (char 0)


In [24]:
# test = get_historical_exchange_rate('USD',  '2024/10/16')
test = get_historical_gold_price('2024/10/16') #* get_historical_exchange_rate('USD', '2024/10/16') /31.1035

print(test)

取得歷史金價時出現錯誤： unhashable type: 'dict'
None


In [22]:
# 手動新增投資交易
def add_transaction():
    global transactions
    # 手動輸入資料
    日期 = input("輸入交易日期 (格式：YYYY-MM-DD): ")
    投資類型 = input("輸入投資類型（Gold 或 Forex）: ")
    投資項目 = input("輸入投資項目（如 Gold、USD、EUR 等）: ")
    
    # 轉換日期格式
    日期 = datetime.strptime(日期, '%Y-%m-%d')
    

    if 投資類型 == 'Gold':
        # 輸入黃金數量
        while True:
            try:
                數量_or_外幣金額 = float(input("輸入黃金數量 (克數): "))
                # 取得當前黃金價格，且由於原本重量以盎司計算將其轉換為克
                gold_price_usd = get_historical_gold_price(日期)/31.1035
                if gold_price_usd is not None:
                    # 獲取匯率
                    exchange_rate = get_historical_exchange_rate('USD', 日期)
                    if exchange_rate is not None:
                        # 將黃金價格轉換為台幣
                        黃金價格 = gold_price_usd * exchange_rate
                        價格 = 黃金價格 * 數量_or_外幣金額
                        break
                print("無法獲取黃金價格，請檢查日期或API設定。")
                return
            except ValueError:
                print("請輸入一個有效的數字。")
    
    elif 投資類型 == 'Forex':
        # 輸入外幣數量
        while True:
            try:
                數量_or_外幣金額 = float(input("輸入買入外幣金額: "))  # 外幣數量由用戶輸入
                # 使用指定日期的匯率
                匯率 = get_historical_exchange_rate(投資項目, 日期)
                if 匯率 is None:
                    print("無法獲取匯率，請檢查日期或API設定。")
                    return
                
                價格 = 匯率 * 數量_or_外幣金額  # 使用匯率計算價格（台幣）
                break
            except ValueError:
                print("請輸入一個有效的數字。")
    
    else:
        print("無效的投資類型！")
        return

    # 建立新交易 DataFrame
    transaction = pd.DataFrame({
        '日期': [日期],
        '投資類型': [投資類型],
        '投資項目': [投資項目],
        '數量_or_外幣金額': [數量_or_外幣金額],
        '買進匯率': [匯率],
        '價格(台幣)': [價格]
    })
    
    transactions = pd.concat([transactions, transaction], ignore_index=True)

# 儲存至 Excel 並追加資料
def save_to_excel():
    if os.path.exists(FILENAME):
        # 如果 Excel 檔案已存在，追加新資料
        with pd.ExcelWriter(FILENAME, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
            transactions.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        # 如果 Excel 檔案不存在，創建新檔案
        transactions.to_excel(FILENAME, index=False)

# 手動新增交易
add_transaction()

# 儲存更新後的投資紀錄
save_to_excel()

print("投資紀錄已儲存至 Excel 檔案。")


ValueError: time data '' does not match format '%Y/%m/%d'